In [66]:
import typing

import pandas as pd

import config
import src

In [67]:
CFG = config.Config()

In [68]:
dataset: pd.DataFrame = pd.read_parquet(CFG.final_data_files["user_content"])
dataset

In [69]:
MODEL: str = "llama3:70b-instruct-q6_K"
CFG.prompt_files.keys()

In [70]:
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["content"], MODEL)
    )
    for label, path in CFG.prompt_files.items()
}

In [73]:
dataset_w_pred = dataset.copy()

for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    dataset_w_pred = dataset_w_pred.join(preds)
    

In [72]:
dataset_w_pred.to_json(f'{CFG.report_dir}/dataset.predicted.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred